In [32]:
] activate ../../Acceleration

 Activating environment at `~/Desktop/Code/Acceleration/Acceleration/Project.toml`


In [52]:
using Acceleration
using LinearAlgebra

n = 1000;  # number of samples
d = 75;    # number of features
selected_labels = [0,1,2];  ## Only two labels allowed

data, labels = load_dataset("MNIST",selected_labels,n,d);
f,∇f,∇ϕ_cjg,σ,L,μ  = linear_regression(data,labels);
μ2 = μ;

┌ Info: Loading dataset MNIST with 60000 samples and 784 features
└ @ Acceleration /Users/victor/Desktop/Code/Acceleration/Acceleration/src/datasets.jl:26


In [53]:
n = size(data,2)
x_ini = zeros(n)

using JuMP
using Ipopt

model = Model(with_optimizer(Ipopt.Optimizer))
set_silent(model)
@variable(model, x[1:n])
@objective(model, Min, f(x))
@constraint(model, con, 0 .<= x)
@time optimize!(model)
x_opt = value.(x);

  0.017087 seconds (3.01 k allocations: 384.500 KiB)


In [54]:
for j in [0.5,1,2,3,4,5,1000]
    
n = length(x_ini)
x = x_ini;
y = zeros(n);
z = zeros(n);
v = zeros(n);
A0 = σ/(2*L);
A = A0;
AX = zeros(n);

k = 2000;
yks = zeros(n,k);
var_ξ = zeros(k)
#μ = 0;
s = zeros(n);
p = 1
ξ = 0;
w = 0;


for i=1:k   
    p = j;
    λ = 0.9*min((L/μ)*(j/i)^2, 1)
    if(p == 1000)
        λ = 1;
    end
    #λ = min((p+1)*(i+1)^(p-1),1)
    a = L-μ; b = -λ*(μ*A + σ); c = -λ*(μ*A^2 + σ*A);
    α = (-b + sqrt(b^2 - 4*a*c)) / (2*a);
    A = A + α;
    x = (A-α)/A*y + α/A*v; 
    AX = AX + α*x;

    η = zeros(n) 
    approx_gradient = ∇f(x)+η;
    
    s = s + α*η;
    z = z - α*approx_gradient;
    
    v_past = v;
    v = max.(0,(z + μ*AX)*inv(1 + μ*(A-A0)));
    y = (A-α)*inv(A)*y + α*inv(A)*v; 
    ξ = sqrt(1-λ)*(μ*(A-α) + σ)/α * norm(v-v_past,2);
    var_ξ[i] = ξ;
    #println(ξ)
    yks[:,i] = y; 
        
end
    
x_out = yks;
p_out = sqrt.(sum((x_out .- x_opt).^2, dims=1)');


filename = string("logs/",j,".txt")
io = open(filename, "w")
for i=1:k
    print(io,i)
    print(io,"\t")
    print(io, p_out[i])
    print(io,"\t")
    println(io, var_ξ[i])
end
close(io)
end